In [1]:
pip install numpy pandas scipy statsmodels

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np

# 讀取數據
df = pd.read_csv("effi_new.txt", delim_whitespace=True, header=None)

# 定義應變數 (Y) 與自變數 (X)
Y = np.log(df[0])  # 成本函數的應變數（對數化）
X = np.log(df.iloc[:, 1:])  # 其他變數（對數化）

# 確保數據格式正確
print(Y.head())
print(X.head())

0    2.772589
1    2.564949
2    2.639057
3    2.772589
4    2.302585
Name: 0, dtype: float64
          1         2         3         4         5         6         7
0  2.890372 -0.839561  1.908371  5.758902  1.068840  2.040610  4.788512
1  2.772589 -1.077928  1.867562  5.521461  0.789775  1.682168  4.627525
2  2.772589 -0.942635  1.895849  5.549076  0.953124  1.831205  4.669791
3  2.890372 -0.897224  1.898893  5.752573  1.001771  1.979221  4.775242
4  2.484907 -2.268184  1.845047  5.036953 -0.423273  0.250681  4.364144


In [7]:
from scipy.optimize import minimize

# 定義 Translog 成本函數
def translog_cost(params, Y, X):
    beta0 = params[0]
    betas = params[1:len(X.columns) + 1]
    sigma_u = params[-2]
    sigma_v = params[-1]

    # SFA 成本函數
    predicted_Y = beta0 + np.dot(X, betas)
    
    # 計算誤差項
    epsilon = Y - predicted_Y

    # 負對數概似函數（MLE 估計）
    log_likelihood = -np.sum(
        np.log(1 / (sigma_u + sigma_v)) - ((epsilon ** 2) / (2 * (sigma_u + sigma_v) ** 2))
    )

    return log_likelihood

In [8]:
# 設定初始參數
init_params = np.ones(len(X.columns) + 3)

# 最小化負對數概似函數
result = minimize(translog_cost, init_params, args=(Y, X), method="L-BFGS-B")

# 取得最佳參數
estimated_params = result.x
print("估計參數：", estimated_params)

估計參數： [ 0.89297625  0.45086241 -1.70161581  0.79982547 -0.0886311  -1.90161767
 -2.24540595  0.25587467 -4.40926108 -4.40926108]


C:\Users\User\AppData\Local\Temp\ipykernel_11912\1235887165.py:18: RuntimeWarning: invalid value encountered in log
  np.log(1 / (sigma_u + sigma_v)) - ((epsilon ** 2) / (2 * (sigma_u + sigma_v) ** 2))


In [11]:
# 計算無效率項（技術無效率）
sigma_u = estimated_params[-2]
sigma_v = estimated_params[-1]

# 計算殘差，這裡的 estimated_params[1:-2] 是用來取自變數的估計值
epsilon = Y - np.dot(X, estimated_params[1:-2])  # 修正這裡

u_hat = np.abs(epsilon) * (sigma_u / (sigma_u + sigma_v))  # 計算無效率
efficiency = np.exp(-u_hat)  # 這樣每個樣本的效率不同

df["efficiency"] = efficiency
df.to_csv("efficiency_results.csv", index=False)

print("技術效率已儲存至 efficiency_results.csv")

技術效率已儲存至 efficiency_results.csv
